In [1]:
%matplotlib inline


LDA Model
=========

Introduces Gensim's LDA model and demonstrates its use on the NIPS corpus.


In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

The purpose of this tutorial is to demonstrate training an LDA model and
obtaining good results.

In this tutorial we will:

* Load data.
* Pre-process data.
* Transform documents to a vectorized form.
* Train an LDA model.

This tutorial will **not**:

* Explain how Latent Dirichlet Allocation works
* Explain how the LDA model performs inference
* Teach you how to use Gensim's LDA implementation in its entirety

If you are not familiar with the LDA model or how to use it in Gensim, I
suggest you read up on that before continuing with this tutorial. Basic
understanding of the LDA model should suffice. Examples:

* `Introduction to Latent Dirichlet Allocation <http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation>`_
* Gensim tutorial: `sphx_glr_auto_examples_core_run_topics_and_transformations.py`
* Gensim's LDA model API docs: :py:class:`gensim.models.LdaModel`

I would also encourage you to consider each step when applying the model to
your data, instead of just blindly applying my solution. The different steps
will depend on your data and possibly your goal with the model.

Data
----

I have used a corpus of NIPS papers in this tutorial, but if you're following
this tutorial just to learn about LDA I encourage you to consider picking a
corpus on a subject that you are familiar with. Qualitatively evaluating the
output of an LDA model is challenging and can require you to understand the
subject matter of your corpus (depending on your goal with the model).

NIPS (Neural Information Processing Systems) is a machine learning conference
so the subject matter should be well suited for most of the target audience
of this tutorial.  You can download the original data from Sam Roweis'
`website <http://www.cs.nyu.edu/~roweis/data.html>`_.  The code below will
also do that for you.

.. Important::
    The corpus contains 1740 documents, and not particularly long ones.
    So keep in mind that this tutorial is not geared towards efficiency, and be
    careful before applying the code to a large dataset.




In [3]:
import io
import os.path
import re
import tarfile

import smart_open

def extract_documents(url='https://cs.nyu.edu/~roweis/data/nips12raw_str602.tgz'):
    fname = url.split('/')[-1]
    
    # Download the file to local storage first.
    # We can't read it on the fly because of 
    # https://github.com/RaRe-Technologies/smart_open/issues/331
    if not os.path.isfile(fname):
        with smart_open.open(url, "rb") as fin:
            with smart_open.open(fname, 'wb') as fout:
                while True:
                    buf = fin.read(io.DEFAULT_BUFFER_SIZE)
                    if not buf:
                        break
                    fout.write(buf)
                         
    with tarfile.open(fname, mode='r:gz') as tar:
        # Ignore directory entries, as well as files like README, etc.
        files = [
            m for m in tar.getmembers()
            if m.isfile() and re.search(r'nipstxt/nips\d+/\d+\.txt', m.name)
        ]
        for member in sorted(files, key=lambda x: x.name):
            member_bytes = tar.extractfile(member).read()
            yield member_bytes.decode('utf-8', errors='replace')

docs = list(extract_documents())

So we have a list of 1740 documents, where each document is a Unicode string. 
If you're thinking about using your own corpus, then you need to make sure
that it's in the same format (list of Unicode strings) before proceeding
with the rest of this tutorial.




In [4]:
print(len(docs))
print(docs[0][:500])

1740
1 
CONNECTIVITY VERSUS ENTROPY 
Yaser S. Abu-Mostafa 
California Institute of Technology 
Pasadena, CA 91125 
ABSTRACT 
How does the connectivity of a neural network (number of synapses per 
neuron) relate to the complexity of the problems it can handle (measured by 
the entropy)? Switching theory would suggest no relation at all, since all Boolean 
functions can be implemented using a circuit with very low connectivity (e.g., 
using two-input NAND gates). However, for a network that learns a pr


Pre-process and vectorize the documents
---------------------------------------

As part of preprocessing, we will:

* Tokenize (split the documents into tokens).
* Lemmatize the tokens.
* Compute bigrams.
* Compute a bag-of-words representation of the data.

First we tokenize the text using a regular expression tokenizer from NLTK. We
remove numeric tokens and tokens that are only a single character, as they
don't tend to be useful, and the dataset contains a lot of them.

.. Important::

   This tutorial uses the nltk library for preprocessing, although you can
   replace it with something else if you want.




In [6]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

We use the WordNet lemmatizer from NLTK. A lemmatizer is preferred over a
stemmer in this case because it produces more readable words. Output that is
easy to read is very desirable in topic modelling.




In [8]:
# Lemmatize the documents.
import nltk
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


We find bigrams in the documents. Bigrams are sets of two adjacent words.
Using bigrams we can get phrases like "machine_learning" in our output
(spaces are replaced with underscores); without bigrams we would only get
"machine" and "learning".

Note that in the code below, we find bigrams and then add them to the
original data, because we would like to keep the words "machine" and
"learning" as well as the bigram "machine_learning".

.. Important::
    Computing n-grams of large dataset can be very computationally
    and memory intensive.




In [9]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

2019-10-25 21:58:37,227 : INFO : collecting all words and their counts
2019-10-25 21:58:37,229 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-10-25 21:58:51,751 : INFO : collected 1120198 word types from a corpus of 4629808 words (unigram + bigrams) and 1740 sentences
2019-10-25 21:58:51,754 : INFO : using 1120198 counts as vocab in Phrases<0 vocab, min_count=20, threshold=10.0, max_vocab_size=40000000>


In [16]:
docs[0]

[u'connectivity',
 u'versus',
 u'entropy',
 u'yaser',
 u'abu',
 u'mostafa',
 u'california',
 u'institute',
 u'of',
 u'technology',
 u'pasadena',
 u'ca',
 u'abstract',
 u'how',
 u'doe',
 u'the',
 u'connectivity',
 u'of',
 u'neural',
 u'network',
 u'number',
 u'of',
 u'synapsis',
 u'per',
 u'neuron',
 u'relate',
 u'to',
 u'the',
 u'complexity',
 u'of',
 u'the',
 u'problem',
 u'it',
 u'can',
 u'handle',
 u'measured',
 u'by',
 u'the',
 u'entropy',
 u'switching',
 u'theory',
 u'would',
 u'suggest',
 u'no',
 u'relation',
 u'at',
 u'all',
 u'since',
 u'all',
 u'boolean',
 u'function',
 u'can',
 u'be',
 u'implemented',
 u'using',
 u'circuit',
 u'with',
 u'very',
 u'low',
 u'connectivity',
 u'using',
 u'two',
 u'input',
 u'nand',
 u'gate',
 u'however',
 u'for',
 u'network',
 u'that',
 u'learns',
 u'problem',
 u'from',
 u'example',
 u'using',
 u'local',
 u'learning',
 u'rule',
 u'we',
 u'prove',
 u'that',
 u'the',
 u'entropy',
 u'of',
 u'the',
 u'problem',
 u'becomes',
 u'lower',
 u'bound',
 u'f

We remove rare words and common words based on their *document frequency*.
Below we remove words that appear in less than 20 documents or in more than
50% of the documents. Consider trying to remove words only based on their
frequency, or maybe combining that with this approach.




In [10]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

2019-10-25 22:04:15,145 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-10-25 22:04:20,267 : INFO : built Dictionary(79429 unique tokens: [u'bshouty', u's2s2s3s4s4s', u'gg9ghhhho0112233', u'homomorphism', u'nordisk']...) from 1740 documents (total 4953968 corpus positions)
2019-10-25 22:04:20,606 : INFO : discarding 70785 tokens: [(u'0a', 19), (u'2h', 16), (u'2h2', 1), (u'2he', 3), (u'__c', 2), (u'_k', 6), (u'a', 1740), (u'about', 1058), (u'abstract', 1740), (u'after', 1087)]...
2019-10-25 22:04:20,608 : INFO : keeping 8644 tokens which were in no less than 20 and no more than 870 (=50.0%) documents
2019-10-25 22:04:20,662 : INFO : resulting dictionary: Dictionary(8644 unique tokens: [u'ma_abstract', u'func', u'four', u'circuitry', u'ac_uk']...)


Finally, we transform the documents to a vectorized form. We simply compute
the frequency of each word, including the bigrams.




In [11]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [18]:
corpus[0:2]

[[(0, 4),
  (1, 2),
  (2, 1),
  (3, 1),
  (4, 2),
  (5, 4),
  (6, 4),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 2),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 8),
  (29, 1),
  (30, 3),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 2),
  (37, 4),
  (38, 2),
  (39, 3),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 3),
  (53, 1),
  (54, 11),
  (55, 1),
  (56, 2),
  (57, 2),
  (58, 12),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 4),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 4),
  (71, 2),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 3),
  (77, 2),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 3),
  (83, 12),
  (84, 1),
  (85, 3),
  (86, 1),
  (87, 2),
  (88, 4),
  (89, 4),
  (90, 3),
  (91,

Let's see how many tokens and documents we have to train on.




In [12]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 8644
Number of documents: 1740


Training
--------

We are ready to train the LDA model. We will first discuss how to set some of
the training parameters.

First of all, the elephant in the room: how many topics do I need? There is
really no easy answer for this, it will depend on both your data and your
application. I have used 10 topics here because I wanted to have a few topics
that I could interpret and "label", and because that turned out to give me
reasonably good results. You might not need to interpret all your topics, so
you could use a large number of topics, for example 100.

``chunksize`` controls how many documents are processed at a time in the
training algorithm. Increasing chunksize will speed up training, at least as
long as the chunk of documents easily fit into memory. I've set ``chunksize =
2000``, which is more than the amount of documents, so I process all the
data in one go. Chunksize can however influence the quality of the model, as
discussed in Hoffman and co-authors [2], but the difference was not
substantial in this case.

``passes`` controls how often we train the model on the entire corpus.
Another word for passes might be "epochs". ``iterations`` is somewhat
technical, but essentially it controls how often we repeat a particular loop
over each document. It is important to set the number of "passes" and
"iterations" high enough.

I suggest the following way to choose iterations and passes. First, enable
logging (as described in many Gensim tutorials), and set ``eval_every = 1``
in ``LdaModel``. When training the model look for a line in the log that
looks something like this::

   2016-06-21 15:40:06,753 - gensim.models.ldamodel - DEBUG - 68/1566 documents converged within 400 iterations

If you set ``passes = 20`` you will see this line 20 times. Make sure that by
the final passes, most of the documents have converged. So you want to choose
both passes and iterations to be high enough for this to happen.

We set ``alpha = 'auto'`` and ``eta = 'auto'``. Again this is somewhat
technical, but essentially we are automatically learning two parameters in
the model that we usually would have to specify explicitly.




In [21]:
id2word

{0: u'2n',
 1: u'_c',
 2: u'a2',
 3: u'a_follows',
 4: u'ability',
 5: u'abu',
 6: u'abu_mostafa',
 7: u'access',
 8: u'accommodate',
 9: u'according',
 10: u'according_to',
 11: u'accumulated',
 12: u'acknowledgement',
 13: u'acknowledgement_this',
 14: u'addison',
 15: u'addison_wesley',
 16: u'afosr',
 17: u'aip',
 18: u'air',
 19: u'air_force',
 20: u'all_possible',
 21: u'although',
 22: u'american',
 23: u'american_institute',
 24: u'amount',
 25: u'an_arbitrary',
 26: u'analog',
 27: u'anyway',
 28: u'ao',
 29: u'appears',
 30: u'appendix',
 31: u'approximately',
 32: u'arbitrary',
 33: u'architecture',
 34: u'are_interested',
 35: u'arise',
 36: u'aspect',
 37: u'assume',
 38: u'assumed',
 39: u'assumption',
 40: u'asymptotic',
 41: u'automaton',
 42: u'available',
 43: u'away',
 44: u'axe',
 45: u'basic',
 46: u'become',
 47: u'becomes',
 48: u'benefit',
 49: u'ber',
 50: u'best',
 51: u'big',
 52: u'binary',
 53: u'biological',
 54: u'bit',
 55: u'bit_per',
 56: u'boolean',
 

In [13]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

2019-10-25 22:06:51,915 : INFO : using autotuned alpha, starting with [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
2019-10-25 22:06:51,925 : INFO : using serial LDA version on this node
2019-10-25 22:06:51,942 : INFO : running online (multi-pass) LDA training, 10 topics, 20 passes over the supplied corpus of 1740 documents, updating model once every 1740 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2019-10-25 22:06:51,944 : INFO : PROGRESS: pass 0, at document #1740/1740
2019-10-25 22:07:11,857 : INFO : optimized alpha [0.06758672, 0.08475909, 0.10017212, 0.04867215, 0.0823116, 0.07340417, 0.06659031, 0.07348999, 0.069364965, 0.10157189]
2019-10-25 22:07:11,873 : INFO : topic #3 (0.049): 0.006*"cell" + 0.005*"image" + 0.005*"tree" + 0.005*"class" + 0.004*"map" + 0.003*"region" + 0.003*"constraint" + 0.003*"node" + 0.003*"neuron" + 0.003*"sample"
2019-10-25 22:07:11,875 : INFO : topic #6 (0.067): 0.010*"image" + 0.004*

2019-10-25 22:07:55,968 : INFO : topic #9 (0.050): 0.008*"classifier" + 0.008*"class" + 0.006*"classification" + 0.005*"node" + 0.004*"matrix" + 0.003*"sample" + 0.003*"sequence" + 0.003*"memory" + 0.003*"distance" + 0.003*"rbf"
2019-10-25 22:07:55,970 : INFO : topic #5 (0.050): 0.009*"layer" + 0.009*"hidden" + 0.008*"recognition" + 0.007*"word" + 0.007*"speech" + 0.006*"net" + 0.005*"trained" + 0.005*"architecture" + 0.004*"hidden_unit" + 0.004*"context"
2019-10-25 22:07:55,972 : INFO : topic #8 (0.062): 0.005*"optimal" + 0.004*"action" + 0.004*"approximation" + 0.004*"bound" + 0.004*"estimate" + 0.004*"policy" + 0.004*"control" + 0.003*"prediction" + 0.003*"class" + 0.003*"sample"
2019-10-25 22:07:55,974 : INFO : topic diff=0.200902, rho=0.377964
2019-10-25 22:07:55,988 : INFO : PROGRESS: pass 6, at document #1740/1740
2019-10-25 22:08:02,835 : INFO : optimized alpha [0.03868795, 0.04395069, 0.042699773, 0.03115356, 0.047269493, 0.050012153, 0.0433922, 0.04615072, 0.06189665, 0.04824

2019-10-25 22:08:33,675 : INFO : topic #3 (0.029): 0.015*"tree" + 0.013*"cell" + 0.012*"map" + 0.012*"orientation" + 0.009*"field" + 0.008*"receptive" + 0.007*"receptive_field" + 0.007*"region" + 0.006*"center" + 0.005*"node"
2019-10-25 22:08:33,677 : INFO : topic #0 (0.037): 0.010*"layer" + 0.009*"threshold" + 0.009*"node" + 0.009*"neuron" + 0.008*"net" + 0.006*"bit" + 0.006*"bound" + 0.005*"theorem" + 0.005*"connection" + 0.004*"hidden"
2019-10-25 22:08:33,679 : INFO : topic #7 (0.047): 0.006*"matrix" + 0.006*"solution" + 0.005*"gradient" + 0.004*"generalization" + 0.004*"noise" + 0.004*"distance" + 0.004*"kernel" + 0.003*"optimal" + 0.003*"dimensional" + 0.003*"xi"
2019-10-25 22:08:33,680 : INFO : topic #5 (0.052): 0.010*"hidden" + 0.009*"recognition" + 0.009*"layer" + 0.008*"word" + 0.008*"speech" + 0.007*"net" + 0.006*"trained" + 0.006*"hidden_unit" + 0.005*"architecture" + 0.004*"context"
2019-10-25 22:08:33,682 : INFO : topic #8 (0.060): 0.006*"action" + 0.006*"optimal" + 0.005*

2019-10-25 22:09:01,944 : INFO : topic #8 (0.059): 0.007*"action" + 0.006*"optimal" + 0.006*"policy" + 0.005*"reinforcement" + 0.004*"control" + 0.004*"prediction" + 0.004*"estimate" + 0.004*"approximation" + 0.004*"bound" + 0.003*"let"
2019-10-25 22:09:01,945 : INFO : topic diff=0.114043, rho=0.235702
2019-10-25 22:09:01,960 : INFO : PROGRESS: pass 17, at document #1740/1740
2019-10-25 22:09:07,350 : INFO : optimized alpha [0.039000735, 0.044152234, 0.042632557, 0.029003998, 0.044210915, 0.057528045, 0.044507593, 0.05254776, 0.05852229, 0.047058154]
2019-10-25 22:09:07,364 : INFO : topic #3 (0.029): 0.015*"map" + 0.014*"cell" + 0.014*"tree" + 0.013*"orientation" + 0.011*"field" + 0.010*"receptive" + 0.009*"receptive_field" + 0.008*"region" + 0.006*"center" + 0.005*"self"
2019-10-25 22:09:07,366 : INFO : topic #0 (0.039): 0.010*"layer" + 0.009*"node" + 0.009*"threshold" + 0.009*"neuron" + 0.008*"net" + 0.007*"bound" + 0.006*"bit" + 0.006*"theorem" + 0.005*"let" + 0.005*"class"
2019-10-

We can compute the topic coherence of each topic. Below we display the
average topic coherence and print the topics in order of topic coherence.

Note that we use the "Umass" topic coherence measure here (see
:py:func:`gensim.models.ldamodel.LdaModel.top_topics`), Gensim has recently
obtained an implementation of the "AKSW" topic coherence measure (see
accompanying blog post, http://rare-technologies.com/what-is-topic-coherence/).

If you are familiar with the subject of the articles in this dataset, you can
see that the topics below make a lot of sense. However, they are not without
flaws. We can see that there is substantial overlap between some topics,
others are hard to interpret, and most of them have at least some terms that
seem out of place. If you were able to do better, feel free to share your
methods on the blog at http://rare-technologies.com/lda-training-tips/ !




In [14]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2019-10-25 22:09:18,298 : INFO : CorpusAccumulator accumulated stats from 1000 documents


Average topic coherence: -1.2219.
[([(0.009883207, u'gaussian'),
   (0.009447834, u'mixture'),
   (0.007822083, u'likelihood'),
   (0.007776968, u'density'),
   (0.0069660144, u'component'),
   (0.0067372774, u'prior'),
   (0.006166739, u'log'),
   (0.0057867425, u'matrix'),
   (0.005542608, u'bayesian'),
   (0.0052381656, u'rule'),
   (0.005175795, u'posterior'),
   (0.004856301, u'hidden'),
   (0.0045446404, u'estimate'),
   (0.0043667345, u'noise'),
   (0.0043658726, u'em'),
   (0.0042184903, u'source'),
   (0.00402033, u'independent'),
   (0.0038268669, u'estimation'),
   (0.0036593245, u'maximum'),
   (0.0036540474, u'approximation')],
  -0.8464928153187588),
 ([(0.018307831, u'neuron'),
   (0.016192446, u'cell'),
   (0.010200098, u'response'),
   (0.009976928, u'stimulus'),
   (0.008517418, u'spike'),
   (0.008286243, u'activity'),
   (0.006442301, u'synaptic'),
   (0.0062103174, u'firing'),
   (0.006186208, u'signal'),
   (0.006078393, u'frequency'),
   (0.0049683326, u'visual')

Things to experiment with
-------------------------

* ``no_above`` and ``no_below`` parameters in ``filter_extremes`` method.
* Adding trigrams or even higher order n-grams.
* Consider whether using a hold-out set or cross-validation is the way to go for you.
* Try other datasets.

Where to go from here
---------------------

* Check out a RaRe blog post on the AKSW topic coherence measure (http://rare-technologies.com/what-is-topic-coherence/).
* pyLDAvis (https://pyldavis.readthedocs.io/en/latest/index.html).
* Read some more Gensim tutorials (https://github.com/RaRe-Technologies/gensim/blob/develop/tutorials.md#tutorials).
* If you haven't already, read [1] and [2] (see references).

References
----------

1. "Latent Dirichlet Allocation", Blei et al. 2003.
2. "Online Learning for Latent Dirichlet Allocation", Hoffman et al. 2010.


